# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,20.44,90,100,2.06,US,1706631681
1,1,leongatha,-38.4833,145.9500,9.92,95,11,0.40,AU,1706631681
2,2,longyearbyen,78.2186,15.6401,-4.09,68,75,6.17,SJ,1706631681
3,3,bilibino,68.0546,166.4372,-33.17,99,100,1.07,RU,1706631682
4,4,bethel,41.3712,-73.4140,2.33,68,100,0.00,US,1706631682


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

---
#### Needed to resinstall hvplot, cartopy, geoviews & pyproj to view map plots

In [6]:
!pip install hvplot

In [12]:
!pip install cartopy
!pip install geoviews
!pip install pyproj

  Obtaining dependency information for cartopy from https://files.pythonhosted.org/packages/41/f4/edfba7b0768930486d4f434183a500de75826aba82e6ffd4884ae2b410b9/Cartopy-0.22.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=1.7 from https://files.pythonhosted.org/packages/28/32/c404f6d5566025b9d94eef0cc21039b0a4c10a392bad2f81edf778b3c2bc/shapely-2.0.2-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
     --------------------------------- ---- 41.0/46.5 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 46.5/46.5 kB 772.3 kB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.1.0 from https://files.pythonhosted.org/packages/89/8f/27350c8fba71a37cd0d316f100fbd96bf139cc2b5ff1ab0dcbc7ac64010a/pyproj-3.6.1-cp310-cp310-win_amd64.whl.metadata
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
    --------------------------------------- 0.2/11.2 MB 4.6 MB/

---

In [32]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(float)

# Configure the map plot
cities_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True,
                                         tiles = "OSM", frame_width = 800, frame_height = 600,
                                         size = humidity, color = "City", alpha = 0.75)

# Display the map
cities_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values

# A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

# Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

# Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,port elizabeth,-33.9180,25.5701,25.61,72,0,2.57,ZA,1706631693
77,77,pisco,-13.7000,-76.2167,26.12,78,0,4.12,PE,1706631700
190,190,temuco,-38.7333,-72.6000,25.87,26,0,2.68,CL,1706631435
260,260,nelson bay,-32.7167,152.1500,23.38,94,0,4.02,AU,1706631728
261,261,hilo,19.7297,-155.0900,24.60,74,0,2.06,US,1706631728
293,293,tamanrasset,22.7850,5.5228,21.95,10,0,3.09,DZ,1706631734
327,327,tazacorte,28.6290,-17.9293,21.54,73,0,1.54,ES,1706631576
354,354,adre,13.4667,22.2000,26.73,12,0,3.89,TD,1706631533
485,485,kyaikkami,16.0800,97.5675,23.60,85,0,2.20,MM,1706631795
515,515,yanchep,-31.5500,115.6833,24.41,67,0,4.04,AU,1706631801


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,port elizabeth,ZA,-33.9180,25.5701,72,
77,pisco,PE,-13.7000,-76.2167,78,
190,temuco,CL,-38.7333,-72.6000,26,
260,nelson bay,AU,-32.7167,152.1500,94,
261,hilo,US,19.7297,-155.0900,74,
293,tamanrasset,DZ,22.7850,5.5228,10,
327,tazacorte,ES,28.6290,-17.9293,73,
354,adre,TD,13.4667,22.2000,12,
485,kyaikkami,MM,16.0800,97.5675,85,
515,yanchep,AU,-31.5500,115.6833,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
pisco - nearest hotel: La Portada
temuco - nearest hotel: Hostal Rayen Centro
nelson bay - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
tamanrasset - nearest hotel: فندق الأمان
tazacorte - nearest hotel: App Leyma
adre - nearest hotel: No hotel found
kyaikkami - nearest hotel: ရှုခင်းသာ
yanchep - nearest hotel: Yanchep Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
36,port elizabeth,ZA,-33.9180,25.5701,72,Waterford Hotel
77,pisco,PE,-13.7000,-76.2167,78,La Portada
190,temuco,CL,-38.7333,-72.6000,26,Hostal Rayen Centro
260,nelson bay,AU,-32.7167,152.1500,94,No hotel found
261,hilo,US,19.7297,-155.0900,74,Dolphin Bay Hotel
293,tamanrasset,DZ,22.7850,5.5228,10,فندق الأمان
327,tazacorte,ES,28.6290,-17.9293,73,App Leyma
354,adre,TD,13.4667,22.2000,12,No hotel found
485,kyaikkami,MM,16.0800,97.5675,85,ရှုခင်းသာ
515,yanchep,AU,-31.5500,115.6833,67,Yanchep Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

humidity = hotel_df["Humidity"].astype(float)

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True,
                                    tiles = "OSM", frame_width = 800, frame_height = 600,
                                    size = humidity, color = "City", alpha = 0.75,
                                    hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Hotel Name,Country)